In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import copy

In [2]:
def harris(filepath, blockSize=2, ksize=3, k=0.004):
    """
    # blockSize (np. 2) - Jest to wielkość sąsiedztwa brana pod uwagę przy wykrywaniu narożników
    blockSize - It is the size of neighbourhood considered for corner detection
    # ksize (np. 3) - Parametr wielkości maksi używanej pochodnej Sobela.
    ksize - Aperture parameter of the Sobel derivative used.
    # k (np. 0.004) - Parametr wolny w równaniu detektora Harris (0.004 – 0.006)
    k - Harris detector free parameter in the equation.
    
    https://docs.opencv.org/3.4/dc/d0d/tutorial_py_features_harris.html
    """
    img = cv2.imread(filepath)
    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_grey = np.float32(img_grey)
    harris = cv2.cornerHarris(img_grey, blockSize, ksize, k)
    img[harris > 0.1 * harris.max()] = [0, 0, 255]
    # co tu się dzieje? co oznacza harris max? zo zwraca harris? czemu to nie jest wyjaśniane.... jprdlsvd kjbsdbkgfjjkdfa
    cv2.imshow('Punkty wykryte detektorem Harrisa', img)
    if cv2.waitKey(0) & 0xff == 27:
        cv2.destroyAllWindows()
    return img  #, harris? jak dostac wsp... 

In [3]:
def harris_extended(filepath, blockSize=2, ksize=3, k=0.004, display_result=False):
    """
    # blockSize (np. 2) - Jest to wielkość sąsiedztwa brana pod uwagę przy wykrywaniu narożników
    blockSize - It is the size of neighbourhood considered for corner detection
    # ksize (np. 3) - Parametr wielkości maksi używanej pochodnej Sobela.
    ksize - Aperture parameter of the Sobel derivative used.
    # k (np. 0.004) - Parametr wolny w równaniu detektora Harris (0.004 – 0.006)
    k - Harris detector free parameter in the equation.
    
    https://docs.opencv.org/3.4/dc/d0d/tutorial_py_features_harris.html
    """
    # Konwersja obrazu do postaci zdjęcia w odcieniach szarości
    zdjecie = cv2.imread(filepath)
    szareZdjecie = cv2.cvtColor(zdjecie, cv2.COLOR_BGR2GRAY) 

    # find Harris corners
    szareZdjecie = np.float32(szareZdjecie)
    harris = cv2.cornerHarris(szareZdjecie, blockSize, ksize, k)
    harris = cv2.dilate(harris, None)
    ret, dst = cv2.threshold(harris, 0.01 * harris.max(), 255, 0)
    dst = np.uint8(dst)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(szareZdjecie, np.float32(centroids), (5, 5), (-1, -1), criteria)

    # Now draw them
    res = np.hstack((centroids, corners))
    res = np.int0(res)
    zdjecie[res[:, 1], res[:, 0]] = [0, 0, 255] # image = cv2.circle(image, (int(v[0]),int(v[1])), radius=2, color=(0, 0, 255), thickness=5)
    zdjecie[res[:, 3], res[:, 2]] = [0, 255, 0]

    if display_result:
        cv2.imshow('Punkty wykryte detektorem Harrisa', zdjecie)
        if cv2.waitKey(0) & 0xff == 27:
            cv2.destroyAllWindows()

    return corners, zdjecie

In [4]:
def harris_extended_in_array(input_array, blockSize=2, ksize=3, k=0.004, threshold=0.01):
    """
    input_array - np.ndarray(dtype="float32") reprezentująca zdjęcie w odcieniach szarości
    # blockSize (np. 2) - Jest to wielkość sąsiedztwa brana pod uwagę przy wykrywaniu narożników
    blockSize - It is the size of neighbourhood considered for corner detection
    # ksize (np. 3) - Parametr wielkości maksi używanej pochodnej Sobela.
    ksize - Aperture parameter of the Sobel derivative used.
    # k (np. 0.004) - Parametr wolny w równaniu detektora Harris (0.004 – 0.006)
    k - Harris detector free parameter in the equation.
    # threshold - odsyłam do dokumentacji
    
    https://docs.opencv.org/3.4/dc/d0d/tutorial_py_features_harris.html
    """
    # find Harris corners
    harris = cv2.cornerHarris(input_array, blockSize, ksize, k)
    harris = cv2.dilate(harris, None)
    ret, dst = cv2.threshold(harris, threshold * harris.max(), 255, 0)
    dst = np.uint8(dst)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(input_array, np.float32(centroids), (5, 5), (-1, -1), criteria)

    return corners, centroids

In [5]:
def fast_stock(filepath):
    img = cv2.imread(filepath)
    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Inicjalizacja detektora FAST z wartościami domyślnymi funkcji
    fast = cv2.FastFeatureDetector.create()
    # Wykrycie punktów charakterystycznych (keypoints)
    kp = fast.detect(img_grey, None)
    # Narysowanie punktów charakterystycznych (keypoints)
    img2 = cv2.drawKeypoints(img_grey, kp, None, color=(255, 0, 0))
    # Wyświetlenie parametrów funkcji
    print("Threshold: ", fast.getThreshold())
    print("nonmaxSuppression: ", fast.getNonmaxSuppression())
    print("neighborhood: ", fast.getType())
    print("Total Keypoints without nonmaxSuppression: ", len(kp))
    cv2.imshow('Punkty wykryte algorytmem Fast', img2)
    if cv2.waitKey(0) & 0xff == 27:
        cv2.destroyAllWindows()
    return img2 #, 

In [6]:
def fast(filepath, threshold=10, nonmaxSuppresion=True, type=cv2.FAST_FEATURE_DETECTOR_TYPE_9_16):
    """
    FAST: Features from  Accelerated  Segment  Test
    cv2.FAST_FEATURE_DETECTOR_TYPE_9_16, cv2.FAST_FEATURE_DETECTOR_TYPE_5_8, cv2.FAST_FEATURE_DETECTOR_TYPE_7_12
    """
    img = cv2.imread(filepath)
    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fast = cv2.FastFeatureDetector.create(threshold, nonmaxSuppresion, type)
    keypoints = fast.detect(img_grey, None)
    img2 = cv2.drawKeypoints(img_grey, keypoints, None, color=(255, 0, 0))
    # cv2.imshow('Punkty wykryte algorytmem Fast', img2)
    if cv2.waitKey(0) & 0xff == 27:
        cv2.destroyAllWindows()
    return img2, keypoints # . . .

In [7]:
def sift(img_filepath, nfeatures=0, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6, display_result=False):
    """
    Parameters
        img_filepath     filepath to RGB image
        nfeatures	The number of best features to retain. The features are ranked by their scores (measured in SIFT algorithm as the local contrast)
        nOctaveLayers	The number of layers in each octave. 3 is the value used in D. Lowe paper. The number of octaves is computed automatically from the image resolution.
        contrastThreshold	The contrast threshold used to filter out weak features in semi-uniform (low-contrast) regions. The larger the threshold, the less features are produced by the detector.
        edgeThreshold	The threshold used to filter out edge-like features. Note that the its meaning is different from the contrastThreshold, i.e. the larger the edgeThreshold, the less features are filtered out (more features are retained).
        sigma	The sigma of the Gaussian applied to the input image at the octave #0. If your image is captured with a weak camera with soft lenses, you might want to reduce the number.
    Note
        The contrast threshold will be divided by nOctaveLayers when the filtering is applied. When nOctaveLayers is set to default (3) and if you want to use the value used in D. Lowe paper, 0.03, set this argument to 0.09.
    
    # https://docs.opencv.org/3.4/da/df5/tutorial_py_sift_intro.html
    # https://docs.opencv.org/4.x/d7/d60/classcv_1_1SIFT.html
    """
    img = cv2.imread(img_filepath)
    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create(nfeatures, nOctaveLayers, contrastThreshold, edgeThreshold, sigma)
    keypoints = sift.detect(img_grey, None)
    zdjecie = cv2.drawKeypoints(img, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    if display_result:
        cv2.imshow(f"Detektor SIFT, {nfeatures=} {nOctaveLayers=} {contrastThreshold=} {edgeThreshold=} {sigma=}", zdjecie)
        if cv2.waitKey(0) & 0xff == 27:
            cv2.destroyAllWindows()
    return zdjecie, keypoints

In [8]:
def sift_in_array(input_array, nfeatures=0, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6, display_result=False):
    """
    Parameters
        input_array     np.ndarray represening an image in greyscale
        nfeatures	The number of best features to retain. The features are ranked by their scores (measured in SIFT algorithm as the local contrast)
        nOctaveLayers	The number of layers in each octave. 3 is the value used in D. Lowe paper. The number of octaves is computed automatically from the image resolution.
        contrastThreshold	The contrast threshold used to filter out weak features in semi-uniform (low-contrast) regions. The larger the threshold, the less features are produced by the detector.
        edgeThreshold	The threshold used to filter out edge-like features. Note that the its meaning is different from the contrastThreshold, i.e. the larger the edgeThreshold, the less features are filtered out (more features are retained).
        sigma	The sigma of the Gaussian applied to the input image at the octave #0. If your image is captured with a weak camera with soft lenses, you might want to reduce the number.
    Note
        The contrast threshold will be divided by nOctaveLayers when the filtering is applied. When nOctaveLayers is set to default (3) and if you want to use the value used in D. Lowe paper, 0.03, set this argument to 0.09.
    
    # https://docs.opencv.org/3.4/da/df5/tutorial_py_sift_intro.html
    # https://docs.opencv.org/4.x/d7/d60/classcv_1_1SIFT.html
    """
    sift = cv2.SIFT_create(nfeatures, nOctaveLayers, contrastThreshold, edgeThreshold, sigma)
    keypoints = sift.detect(input_array, None)
    just_coords = [p.pt for p in kp]
    zdjecie = cv2.drawKeypoints(input_array, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    if display_result:
        cv2.imshow(f"Detektor SIFT, {nfeatures=} {nOctaveLayers=} {contrastThreshold=} {edgeThreshold=} {sigma=}", zdjecie)
        if cv2.waitKey(0) & 0xff == 27:
            cv2.destroyAllWindows()
    return keypoints, just_coords, zdjecie

In [9]:
def mouse_callback_function(event, x, y, flags, param):
    global pixel_coords_4_points  # lista globalna
    if event == cv2.EVENT_LBUTTONDOWN:  # cv2.EVENT_LBUTTONDOWN EVENT_LBUTTONDBLCLK
        pixel_coords_4_points.append((x, y))
        print(pixel_coords_4_points)
    if len(pixel_coords_4_points)==4:
        cv2.destroyAllWindows()

In [10]:
def warpPoints(list_of_points, transformation_matrix):
    """
    warpPerspective for list of 2D points
    Inspired by https://docs.opencv.org/3.4/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87

    Args:
        list_of_points (list): list of 2-element tuples eg.: [(0,0), (-2,-4), (7,7)]
        transformation_matrix (np.ndarray): matrix used for transoformation according to equation in https://docs.opencv.org/3.4/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87

    Returns:
        list_of_transformed_points
    """
    list_of_transformed_points = []
    for p in list_of_points:
        tp = ((transformation_matrix[0,0]*p[0] + transformation_matrix[0,1]*p[1] + transformation_matrix[0,2])/(transformation_matrix[2,0]*p[0] + transformation_matrix[2,1]*p[1] + transformation_matrix[2,2]),
              (transformation_matrix[1,0]*p[0] + transformation_matrix[1,1]*p[1] + transformation_matrix[1,2])/(transformation_matrix[2,0]*p[0] + transformation_matrix[2,1]*p[1] + transformation_matrix[2,2]))
        list_of_transformed_points.append(tp)
    return list_of_transformed_points

# Dane wejściowe (folder ze zdjęciami)
i przygotowanie list z macierzami reprezentującymi zdjęcia (rgb/grey, orginalny_rozmiar/resize)

In [11]:
# Wczytanie z folderu obrazków w rgb i obrazków skali szarości (jako macierze) do list: images i images_grey
board_images_folder = r"C:\SEM6\PCPO\p9\images_2"
# board_images_folder = r"C:\SEM6\PCPO\p9\images"
# board_images_folder = r"D:\SEMESTR6\PCPO\p8\board_images"
files = [f for f in listdir(board_images_folder) if isfile(join(board_images_folder, f))]
images = []
images_grey = []
for filename in files:
    filepath = rf"{board_images_folder}\\{filename}"
    img = cv2.imread(filepath)
    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    images.append(img)
    images_grey.append(img_grey)

In [12]:
# Resize obrazów i wczytanie ich do list: images_resized i images_grey_resized
images_resized = []
images_grey_resized = []
scale_percent = 70 # percent of original size
for img_grey, img in zip(images_grey, images):
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    img_grey_resized = cv2.resize(img_grey, dim, interpolation=cv2.INTER_CUBIC)
    img_resized = cv2.resize(img, dim, interpolation=cv2.INTER_CUBIC)
    images_grey_resized.append(img_grey_resized)
    images_resized.append(img_resized)

In [13]:
# Wyświetlenie obrazków z wybranej listy
lista = images_resized
for i, img in enumerate(lista):
    cv2.imshow(f'image {i+1} out of {len(lista)}', img)
    if cv2.waitKey(0) & 0xff == 27:
        cv2.destroyAllWindows()

# test image
czyli przejście przez proces generowania i znajdowania punktów dla przykładowego zdjęcia

In [14]:
# Wybranie zdjęcia 
test_img = images_grey_resized[0]
# test_img

In [15]:
 # Klikamy cztery narożniki tworzące prostokąt (zawsze dokładnie te same na każdym zdjęciu!) w ustalonej kolejkności: LU RU RD LD
pixel_coords_4_points = [] 
cv2.namedWindow("Click four corners")
cv2.setMouseCallback("Click four corners", mouse_callback_function)
cv2.imshow("Click four corners", test_img)
if cv2.waitKey(0):
    cv2.destroyAllWindows()

# Kliknięte współrzędne pikselowe zapisywane są w liście pixel_coords_4_points
pixel_coords_4_points = np.array(pixel_coords_4_points, dtype=np.dtype('float32'))
# pixel_coords_4_points

[(409, 296)]
[(409, 296), (925, 298)]
[(409, 296), (925, 298), (893, 963)]
[(409, 296), (925, 298), (893, 963), (470, 941)]


In [16]:
# Tworzymy nasz układ współrzędnych (wybieramy cztery punkty odpowiadające klikniętym)
square_side = 4.8  # długość boku jednego kwadratu (pola szachownicy)
squares_w = 15  # szerokość w kwadratach prostokąta klikanego (klikanych czterech narożników)
squares_h = 23  # wysokość w kwadratach prostokąta klikanego (klikanych czterech narożników)
half_square_side = square_side / 2
our_coords_4_points = [(0, 0), (square_side*squares_w, 0), (square_side*squares_w, square_side*squares_h), (0, square_side*squares_h)]  # 15 x 23
our_coords_4_points = np.array(our_coords_4_points, dtype=np.dtype('float32'))
# our_coords_4_points

In [17]:
# Macierz do transformacji z układu wsp. pikselowych do naszego układu wsp.
macierz_transformacji = cv2.getPerspectiveTransform(pixel_coords_4_points, our_coords_4_points)

# Macierz do odwrotnej transformacji (z naszego układu wsp. do układu wsp. pikselowych)
macierz_odwrotna = np.linalg.inv(macierz_transformacji)  

In [18]:
# Robimy siatkę/listę punktów w naszym układzie współrzędnych
x = np.arange(0, squares_w*square_side+square_side, square_side)
y = np.arange(0, squares_h*square_side+square_side, square_side)
xx, yy = np.meshgrid(x, y)
our_xyz_points = np.hstack([np.expand_dims(xx.ravel(),axis=0).T, np.expand_dims(yy.ravel(),axis=0).T, np.expand_dims(np.repeat(1, yy.size),axis=0).T]) 
our_xy_points = our_xyz_points[:, :2]
# our_xy_points

In [19]:
# Transformacja siatki z naszego układu do układu pikselowego zdjęcia
our_xy_points_transformed = warpPoints(our_xy_points, macierz_odwrotna)
# our_xy_points_transformed

In [20]:
# Wyświetlenie gridu w układzdie pikselowym
image = copy.deepcopy(test_img) # test_img = images_grey_resized[0]
for v in our_xy_points_transformed:
    image = cv2.circle(image, (int(v[0]),int(v[1])), radius=2, color=(0, 255, 255), thickness=5)
cv2.imshow('Grid (our xyz points transformed)', image)
if cv2.waitKey(0) & 0xff == 27:
    cv2.destroyAllWindows()

In [21]:
# Wykrycie punktów narożnych detektorem harrisa
detected_points, centroids = harris_extended_in_array(test_img, threshold=0.01)

In [22]:
# Wyświetlenie wykrytych narożników
image = copy.deepcopy(test_img) # test_img = images_grey_resized[0]
for v in detected_points:
    image = cv2.circle(image, (int(v[0]),int(v[1])), radius=2, color=(0, 0, 255), thickness=5)
cv2.imshow('Detected points (harris detector)', image)
if cv2.waitKey(0) & 0xff == 27:
    cv2.destroyAllWindows()

In [23]:
# Transformacja wykrytych narożników do naszego układu współrzędnych
detected_points_in_our_coords = warpPoints(detected_points, macierz_transformacji)
# detected_points_in_our_coords

In [24]:
# Dla każdego punktu z naszej siatki wykrycie najbliższego punktu wykrytego detektorem 
closest_detected_points_in_our_coords = []
defects = []
for p in our_xy_points:
    distances = np.linalg.norm(detected_points_in_our_coords - p, axis=1)
    min_index = np.argmin(distances)
    defects.append(distances[min_index]>half_square_side)
    closest_detected_points_in_our_coords.append(detected_points_in_our_coords[min_index])
    # print(f"the closest point to {p} is {detected_points_in_our_coords[min_index]}, at a distance of {distances[min_index]}")
# closest_detected_points_in_our_coords

# Odległości raczej powinny się zgadzać, nie wiem co się stanie, jeśli będą pary punktów o dużej odległości między punktami
if any(defects):
    print(f"OOPS, there is a point from our grid that has the closest point detected by detector further away than {half_square_side}")
    raise Exception("WHAT NOW?")

In [29]:
# Transformacja punktów z detektora (najbliższych siatce) do ich orginalnego pikselowego układu współrzędnych zdjęcia
closest_detected_points = warpPoints(closest_detected_points_in_our_coords, macierz_odwrotna)
closest_detected_points

[(409.37368774414057, 296.388427734375),
 (442.88012695312506, 296.5984191894532),
 (476.40658569335943, 296.7171325683594),
 (509.68069458007807, 296.8659973144531),
 (543.6087646484375, 297.1778869628906),
 (577.3048095703125, 297.3700866699218),
 (611.1991577148438, 297.4745788574218),
 (645.4179077148436, 297.47241210937494),
 (679.6614379882811, 297.50146484375006),
 (714.468994140625, 297.47946166992193),
 (749.3787841796875, 297.46514892578125),
 (784.3598022460936, 297.4997863769531),
 (819.4753417968749, 297.53149414062494),
 (854.7652587890626, 297.4965209960938),
 (890.5591430664061, 297.55526733398443),
 (926.5804443359374, 297.6546936035156),
 (412.84457397460943, 330.3744506835938),
 (445.8460998535156, 330.60891723632807),
 (478.91882324218744, 330.78240966796875),
 (511.780029296875, 331.30523681640625),
 (545.5054321289064, 331.447998046875),
 (578.9163208007812, 331.56530761718756),
 (612.5162353515625, 331.6110229492187),
 (646.4988403320312, 331.7615966796875),
 (68

In [32]:
our_xy_points_transformed

[(408.99999999999994, 296.0),
 (442.20087299397824, 296.12868555424023),
 (475.56749116315757, 296.2580135316402),
 (509.1010987612414, 296.38798875488857),
 (542.8029525273729, 296.5186160950674),
 (576.6743218431343, 296.6499004722602),
 (610.7164888919237, 296.7818468561702),
 (644.930748820745, 296.9144602667471),
 (679.3184099044611, 297.0477457748235),
 (713.8807937125493, 297.1817085027618),
 (748.6192352784061, 297.31635362511014),
 (783.5350832712431, 297.4516863692684),
 (818.6297001706275, 297.58771201616526),
 (853.9044624437067, 297.7244359009446),
 (889.3607607251706, 297.86186341366346),
 (925.0, 298.0),
 (412.1976266325503, 329.8109701310649),
 (445.0952495748287, 330.0218461112618),
 (478.1555609341951, 330.2337649351766),
 (511.3797705151463, 330.44673435773774),
 (544.7691001473248, 330.6607622109555),
 (578.3247838352996, 330.8758564048823),
 (612.0480679105914, 331.09202492858725),
 (645.940211185981, 331.3092758511451),
 (680.0024851121424, 331.52761732264037),
 (

# TODO: Robimy to samo co wyżej dla test image, tylko w pętli dla każdego z 12 zdjęć

Można zrobić jedną dużą funkcję i wrzucić do niej to wszystko co wyżej.

Funkcja ta oprócz macierzy/zdjęcia na wejściu będzie mogła przyjmować też parametry takie jak square_side, squares_w, squares_h, ...

Na wyjściu daje dwie listy (our_xy_points_transformed, closest_detected_points).

Potem można zrobić kolejną funkcję przyjmującą na wejściu folder ze zdjęciami i wykonującą tą dużą funkcję dla każdego zdjęcia w folderze (i będzie zwracać np. słownik z parami list)

TODO:
- done 1. stworzyć grid (siatkę) współrzędnych punktów przecięcia kwadratów w naszym układzie współrzędnych (może wychodzic poza obrazek, np +100 **punktów** w górę, - 100 w dół, +100 prawo, -100 lewo) 
- done 2. zadziałać na tych współrzędnych macierzą odwrotną (wzór w docach opencv, funkcja getPerspectiveTransform), dostaniemy wtedy współrzędne tych punktów w układzie pikselowym orginalnego zdjęcia
- done 3. wykryć na orginalnym zdjęciu punkty np. detektorem Harrisa, punkty nie muszą być tylko na "narożnikach" (punktach przecięcia kwadratów)
- done 4. dla każdego z punktów siatki w układzie pikselowym zdjęcia (punkty powstałe po kroku 2.) znajdujemy taki punkt (współrzędne) spośród wykrytych punktów, którego odległość jest najbliższa temu pierwszemu
- done 5. czyli mamy dwie listy współrzędnych punktów, potem będziemy to wykorzystywać do kalibracji kamery.
- TODO 6. zrobić ten proces dla każdego z 12 zdjęć i zapisywać w jakieś strukturze danych dla każdego zdj dwie listy punktów: our_xy_points_transformed i closest_detected_points (można zrobić słownik, gdzie kluczem będą nazwy/numery zdjęć)